# Emotion Prediction from RGB Data Using CNN

This Jupyter Notebook demonstrates how to predict emotions from RGB data using a Convolutional Neural Network (CNN) in Keras. We will use this demmo data generated to simulate RGB values associated with different emotional states.

## Table of Contents
1. Importing Libraries
2. Loading and Preparing the Data
3. Data Preprocessing
4. Building the CNN Model
5. Training the Model
6. Evaluating the Model
7. Conclusion and Further Steps

## 1. Importing Libraries

In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import kerastuner as kt
from tensorflow import keras

## 2. Loading and Preparing the Data

In [9]:
#Initial form of the data 
data = pd.read_csv('demmo_data.csv')

# Preprocess RGB columns
data['RGB_1'] = data['RGB_1'].apply(lambda x: eval(x))
data['RGB_2'] = data['RGB_2'].apply(lambda x: eval(x))
data['RGB_3'] = data['RGB_3'].apply(lambda x: eval(x))

# Extract features and labels
X = np.hstack([np.vstack(data['RGB_1']), np.vstack(data['RGB_2']), np.vstack(data['RGB_3'])])
y = data['Emotional_Word']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data to fit the CNN model
X_train_cnn = X_train.reshape(-1, 3, 3, 1)
X_test_cnn = X_test.reshape(-1, 3, 3, 1)

# Print the shapes to verify
print("Shape of X_train_cnn:", X_train_cnn.shape)
print("Shape of X_test_cnn:", X_test_cnn.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train_cnn: (182, 3, 3, 1)
Shape of X_test_cnn: (46, 3, 3, 1)
Shape of y_train: (182, 155)
Shape of y_test: (46, 155)


### Step 3: Define the CNN Model with Keras Tuner
#### Adjust the input shape in the CNN model to (3, 3, 1).

In [10]:
def build_simple_cnn_model():
    model = Sequential()
    model.add(Input(shape=(3, 3, 1)))  # Reshape input to 3x3x1 (as a pseudo-image)
    
    # First Convolutional Layer
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    # Second Convolutional Layer
    model.add(Conv2D(64, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())  # Flatten the output for the dense layer
    
    # Dense Layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer
    
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Build and summarize the model
simple_cnn_model = build_simple_cnn_model()
simple_cnn_model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 2, 2, 32)       │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1, 1, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 0, 0, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 0, 0, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 0, 0, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 155)            │        19,995 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 28,539 (111.48 KB)

 Trainable params: 28,539 (111.48 KB)

 Non-trainable params: 0 (0.00 B)

### Step 4: Initialize Keras Tuner and Search for Best Hyperparameters

In [11]:
# Train the simple CNN model
history = simple_cnn_model.fit(
    X_train_cnn,
    y_train,
    epochs=50,
    validation_data=(X_test_cnn, y_test),
    callbacks=[keras.callbacks.EarlyStopping(patience=10)]
)

# Evaluate the model
loss, accuracy = simple_cnn_model.evaluate(X_test_cnn, y_test)
print(f'Test Accuracy: {accuracy}')


Epoch 1/50


ValueError: Exception encountered when calling Conv2D.call().

[1mNegative dimension size caused by subtracting 2 from 1 for '{{node sequential_1_1/conv2d_3_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](sequential_1_1/dropout_3_1/stateless_dropout/SelectV2, sequential_1_1/conv2d_3_1/convolution/ReadVariableOp)' with input shapes: [?,1,1,32], [2,2,32,64].[0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(None, 1, 1, 32), dtype=float32)

## 4. Building the CNN Model
##### I use a VGG-16-like architecture adapted for my dataset

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()

# Convolutional layers
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(3, 3, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu' , padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.summary()


c:\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 1, 1, 64)       │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 0, 0, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 0, 0, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 0, 0, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 0, 0, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 0, 0, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 77)             │        39,501 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 115,661 (451.80 KB)

 Trainable params: 115,661 (451.80 KB)

 Non-trainable params: 0 (0.00 B)

## 5. Training the Model


In [12]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_val, y_val))


Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 3, 3), dtype=float32). Expected shape (None, 3, 3, 3), but input has incompatible shape (None, 3, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 3, 3), dtype=float32)
  • training=True
  • mask=None

## 6. Evaluating the Model

In [11]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Making predictions
def predict_emotion(rgb_combination):
    rgb_combination = np.array(rgb_combination).reshape((1, 3, 3, 1))
    prediction = model.predict(rgb_combination)
    return label_encoder.inverse_transform([np.argmax(prediction)])[0]

# Example prediction
sample_rgb_combination = [[251/255.0, 167/255.0, 157/255.0], [255/255.0, 242/255.0, 124/255.0], [179/255.0, 22/255.0, 61/255.0]]
print(predict_emotion(sample_rgb_combination))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.0000e+00 - loss: 6.3176
Test Accuracy: 0.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Pretty


## 7. Conclusion and Further Steps
